<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import json
import csv
from keras import layers,Input
from keras.models import Model
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd

# Stage 1 資料前處理

## 讀入向量化資料


In [24]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

In [25]:
len(data['1'])

764

## 讀入 True label csv

In [26]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/TrainLabel.csv'
response = urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
true_labels = list(csv.reader(lines))
del true_labels[0]

In [47]:
len(true_label])

2

## 建立 False label 資料

In [28]:
import random

input_keys = list(data.keys())
false_labels = []

while len(false_labels) < len(true_labels):
  temp = [random.choice(input_keys),random.choice(input_keys)]
  if temp not in true_labels:
    false_labels.append(temp)

In [29]:
len(false_labels)

1381

## 產生所需資料格式

In [30]:
def data_generator(input,Label=0):
  return [[[data[row[0]],data[row[1]]],Label] for row in input]

In [31]:
# 標籤為True的向量資料對
true_list = data_generator(true_labels,1)

In [32]:
# 標籤為False的向量資料對
false_list = data_generator(false_labels,0)

In [33]:
df_true = pd.DataFrame(true_list)
df_true.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [34]:
df_false = pd.DataFrame(false_list)
df_false.head()

,0,1
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,"[[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0


## 混合 Ture / False 標籤的資料

In [35]:
train_list = true_list+false_list

In [36]:
len(train_list)

2762

In [37]:
df_train = pd.DataFrame(train_list)
df_train.head()

,0,1
0,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
1,"[[0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [38]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)

In [39]:
df_train.head()

,0,1
690,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
1559,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 2,...",0
1586,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1628,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",0
764,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,...",1


In [40]:
X = df_train[0]
Y = df_train[1]

## 區分訓練集與驗證集

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_vaild, y_train, y_vaild = train_test_split(X, Y, test_size=.25)

In [ ]:
x_train[0]

In [42]:
len(x_train)

2071

In [43]:
len(x_vaild)

691

### 轉為Numpy陣列

In [69]:
left_x_train = np.array([row[0] for row in x_train])
right_x_train = np.array([row[1] for row in x_train])
left_x_vaild = np.array([row[0] for row in x_vaild])
right_x_vaild = np.array([row[1] for row in x_vaild])
y_train = np.array(y_train)
y_vaild = np.array(y_vaild)

In [73]:
pd.DataFrame(left_x_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
pd.DataFrame(right_x_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 建立神經網路模型

參考資料：
* [Image similarity estimation using a Siamese Network with a triplet loss](https://keras.io/examples/vision/siamese_network/)
* [Siamese Network — Unofficial Keras Implementation](https://github.com/gchoi/face-recognition-using-siamese-network)
* [Text-Similarity-Using-Siamese-Deep-Neural-Network](https://github.com/AnjaliDharmik/Text-Similarity-Using-Siamese-Deep-Neural-Network)

## 建構原理
藉由同一個層來一起進行萃取兩篇文章的相似度特徵  
因此需共享權重來訓練神經網路

In [92]:
lstm = layers.LSTM(382) # 共享權重的神經層

left_input = Input(shape=(None,),name='left_input')
print(left_input.shape)
left_embedded_text = layers.Embedding(2071,764)(left_input)
left_output = lstm(left_embedded_text)
print(left_output.shape)

right_input = Input(shape=(None,),name='right_input')
print(right_input.shape)
right_embedded_text = layers.Embedding(2071,764)(right_input)
right_output = lstm(right_embedded_text)
print(right_output.shape)

merged = layers.concatenate([left_output,right_output],axis=-1) # 將向量串接
print(merged.shape)

predictions = layers.Dense(1,activation='sigmoid')(merged)
model = Model([left_input,right_input],predictions)

(None, None)
(None, 382)
(None, None)
(None, 382)
(None, 764)


In [93]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 right_input (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, None, 764)    1582244     ['left_input[0][0]']             
                                                                                                  
 embedding_6 (Embedding)        (None, None, 764)    1582244     ['right_input[0][0]']            
                                                                                           

In [94]:
model = Model([left_input,right_input],predictions)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'left_input':left_x_train,'right_input':right_x_train},y_train,epochs=10,batch_size=128)

Epoch 1/10
 3/17 [====>.........................] - ETA: 11:04 - loss: 0.0000e+00 - acc: 0.5234